## Matrix representation of operators

We saw briefly that we can represent finite difference operators as matrices. Let's look at this in a bit more detail.

To do this, we need to provide an ordering of all of the degrees of freedom (dofs) in our finite difference discretisation. In one dimension, we order the points in the domain from left to right and use a single index:

$$
x_0 < x_1 < \dots < x_{n-1}
$$

and so we have a single index for all the points $i = [0, 1, \dots, n-1]$. We can therefore represent our function $u(x)$ discretised at the points $\{x_i\}$ as a vector in $\mathbb{R}^n$

$$
U = \begin{bmatrix} u_0 \\ u_1 \\ \vdots \\ u_{n-1} \end{bmatrix}
$$

and similarly with the right hand side $f(x)$. The differencing operators *combine* entries from $U$ linearly to produce a new vector $D U$. Since this operation is linear, we can represent it as a matrix

$$
D : \mathbb{R}^n \to \mathbb{R}^n
$$

which takes in a vector $U$ and spits out a new vector representing the action of the differencing operator on $U$.

For example, the left-looking operator $D_- u_i = \frac{u_i - u_{i-1}}{h}$ uses, at each point $i$ values from points $i$ and $i-1$. On a grid with 4 points, this can be represented as the matrix

$$
D_- = \frac{1}{h}
\begin{bmatrix}
1 & 0 & 0 & 0\\
-1 & 1 & 0 & 0\\
0 & -1 & 1 & 0\\
0 & 0 & -1 & 1
\end{bmatrix}.
$$

Similarly, the centered difference approximation of $\frac{\text{d}^2}{\text{d} x^2}$, $D^2 u_i = \frac{u_{i+1} - 2u_i + u_{i-1}}{h^2}$ can be written

$$
D^2 = \frac{1}{h^2}
\begin{bmatrix}
-2 & 1 & 0 & 0\\
1 & -2 & 1 & 0\\
0 & 1 & -2 & 1\\
0 & 0 & 1 & -2
\end{bmatrix}.
$$

### "Matrix-free" implementation

If we only never need to apply the differencing operator, it might make sense (memory or efficiency, for example) to just provide a function which computes the matrix-vector multiplication without storing the matrix. Let's see this in action.

In [ ]:
%matplotlib notebook
from matplotlib import pyplot
import numpy

def dminus(u, h):
    n, = u.shape
    du = numpy.zeros_like(u)
    for i in range(n):
        if i == 0:
            du[i] = 1/h * u[i]
        else:
            du[i] = 1/h * (u[i] - u[i-1])
    return du

def dminusop(u, h):
    n, = u.shape
    D = numpy.eye(n) - numpy.diag(numpy.full(n-1, 1), k=-1)
    D *= 1/h
    return D

In [ ]:
n = 10
u = numpy.random.rand(n)
h = 1/n
dminus(u, h)

In [ ]:
D = dminusop(u, h)

In [ ]:
D @ u

In [ ]:
numpy.allclose(D@u, dminus(u, h))

Which one is faster? Let's have a go with a bigger grid. We can use notebook "magic" `%%timeit` to time the execution of a cell.

In [ ]:
n = 10000
u = numpy.random.rand(n)
h = 1/n

In [ ]:
%%timeit
dminus(u, h)

In [ ]:
D = dminusop(u, h)

In [ ]:
%%timeit
D @ u

Perhaps surprisingly, the python loops are faster than the numpy matrix-vector product. This is likely because the numpy matrix is 10000 x 10000 and dense (and we do a lot of work multiplying by zero). We should probably use a *sparse* matrix (see below).

We can also attempt to speed up the loop by using the Python JIT compiler [numba](https://numba.pydata.org) (available via `pip install numba`).

In [ ]:
import numba

@numba.jit
def dminus_compiled(u, h):
    n, = u.shape
    du = numpy.zeros_like(u)
    for i in range(n):
        if i == 0:
            du[i] = 1/h * u[i]
        else:
            du[i] = 1/h * (u[i] - u[i-1])
    return du

In [ ]:
%%timeit
dminus_compiled(u, h)

Nearly a 1000x speedup. This doesn't work for all functions, but if you have code with loops and numpy arrays, it's probably worth a shot.

## 2D finite differences

Now, finally, let's look at finite differences in 2D. We remind ourselves of the differential operators we might encounter. Rather than just a derivative in the $x$ direction, we can take derivatives of a function in both $x$ and $y$.

$$
\begin{aligned}
\partial_x u &= \frac{\partial u(x, y)}{\partial x}\\
\partial_y u &= \frac{\partial u(x, y)}{\partial y}
\end{aligned}.
$$

Often we see vector-calculus operators.

### Gradient

For a scalar $u(x, y)$ the 2D gradient is a vector

$$
\nabla u(x, y) := \begin{bmatrix} \partial_x u \\ \partial_y u \end{bmatrix}.
$$

### Divergence

For a vector $\vec{w}(x, y) = \begin{bmatrix} w_0 \\ w_1 \end{bmatrix}$, the divergence is a scalar

$$
\nabla \cdot \vec{w} = \partial_x w_0 + \partial_y w_1.
$$

### Laplacian

For a scalar $u(x, y)$ the Laplacian is a scalar

$$
\nabla^2 u(x, y) := \nabla \cdot \nabla u(x, y) = \partial_x^2 u + \partial_y^2 u.
$$

### Finite difference operators

As usual, we need some domain $\Omega$ in which we will solve the problem. Given some domain, we need to choose a way of specifying it, and ordering the degrees of freedom. This is very fiddly for anything other than coordinate aligned rectangular domains (one of the major disadvantages of finite differences). As a result, all of the problems we will solve will be on squares and rectangles.

Lets choose $\Omega = (0, W) \times (0, H)$. We'll pick $N_x$ points in the x-direction, and $N_y$ in the y-direction. We'll choose a typewriter ordering of degrees of freedom (bottom-to-top, left-to-right), so given an index $i$ in the x-direction and an index $j$ in the y-direction it represents the point

$$
(x, y) = (i h_x, j h_y)
$$

where

$$
\begin{aligned}
h_x &= \frac{W}{N_x - 1}\\
h_y &= \frac{H}{N_y - 1}\\
\end{aligned}
$$

and $i \in \{0, N_x - 1\}$, $j \in \{0, N_y - 1\}$.

We will again represent our solution vectors as 1D vectors (remembering that we should plot them in 2D).

Let's write some code to encapsulate a domain and draw vectors.

In [ ]:
from collections import namedtuple
Point = namedtuple("Point", ("x", "y"))
class Grid(object):
    def __init__(self, Nx, Ny, P0, P1):
        X0, Y0 = P0
        X1, Y1 = P1
        self.W = X1 - X0
        self.H = Y1 - Y0
        self.Nx = Nx
        self.Ny = Ny
        x = numpy.linspace(X0, X1, self.Nx)
        y = numpy.linspace(Y0, Y1, self.Ny)
        self.XY = numpy.meshgrid(x, y, indexing="ij")
        
    @property
    def hx(self):
        return self.W/(self.Nx - 1)
    
    @property
    def hy(self):
        return self.H/(self.Ny - 1)

    def alpha(self, i, j):
        return i*self.Ny + j

    def new_vector(self, components=1):
        vec = numpy.zeros(self.Nx*self.Ny*components, dtype=float)
        shape = (self.Nx, self.Ny)
        if components > 1:
            shape = shape + (components, )
        return vec.reshape(shape)
    
    def contourf(self, u):
        U = u.reshape(self.Nx, self.Ny)
        pyplot.figure()
        
        pyplot.contourf(*self.XY, U)
        pyplot.colorbar()
        
    def quiver(self, u, colour=None):
        U = u.reshape(self.Nx, self.Ny, 2)
        pyplot.figure()
        if colour is None:
            pyplot.quiver(*self.XY, U[..., 0], U[..., 1])
        else:
            pyplot.quiver(*self.XY, U[..., 0], U[..., 1], colour)

In [ ]:
grid = Grid(17, 15, P0=Point(-2, -1), P1=Point(1, 1))

In [ ]:
X, Y = grid.XY
u = grid.new_vector(components=2)
u[..., 0] = -Y
u[..., 1] = X

In [ ]:
grid.quiver(u)

Notice how we return vectors that we can index with two indices (or three if we have a vector). For 2D indexing of a vector, I'll write (using roman indices):

$$
U_{i, j}
$$

to indicate the value at $(i h_x, j h_y)$.

We can translate these 2D indices into a 1D index to a flat vector. I'll use greek letters for these flat indices.

$$
\alpha(i, j) := i N_y + j
$$

Now let's think about solving an equation, we'll start by solving the 2D Laplacian with Dirichlet conditions.

$$
\begin{aligned}
-\nabla^2 u &= f && \text{ on }\Omega = (0, 1) \times (0, 1)\\
         u &= g && \text{ on }\partial\Omega
\end{aligned}
$$

We'll pick $f = 8\pi^2\sin(2\pi x)\sin(2\pi y)$ and set $g = 0$.

Since we're only doing things on axis-aligned domains, the derivatives decompose into directional derivatives, and so the 2D stencil is just the "sum" of the two 1D stencils for $\partial_x^2$ and $\partial_y^2$. Note that we must be careful to use the correct $h_x$ or $h_y$.

So we have

$$
-\nabla^2 = \frac{1}{h_x^2} \begin{bmatrix} & & \\ -1 & 2 & -1 \\ & & \end{bmatrix} + \frac{1}{h_y^2} \begin{bmatrix} & -1 & \\ & 2 & \\ & -1 & \end{bmatrix}.
$$

Where this stencil notation is to be understood as being laid over the 2D grid. We will come to the indexing in a moment.

### Indexing matrix representation in 2D
### Sparse matrices

## CFL conditions

In [ ]:
%matplotlib notebook
import pylab as plt
import numpy
from matplotlib import animation, rc

from IPython.display import HTML

rc('animation', html='jshtml')

# Generate data for plotting
Lx = Ly = 3
Nx = Ny = 11
Nt = 20
x = numpy.linspace(0, Lx, Nx)
y = numpy.linspace(0, Ly, Ny)
x,y = numpy.meshgrid(x,y)
z0 = numpy.exp(-(x-Lx/2)**2-(y-Ly/2)**2)   # 2 dimensional Gaussian

def some_data(i):   # function returns a 2D data array
    return z0 * (i/Nt)

fig = plt.figure()
ax = plt.axes(xlim=(0, Lx), ylim=(0, Ly), xlabel='x', ylabel='y')

cvals = numpy.linspace(0,1,Nt+1)      # set contour values 
cont = plt.contourf(x, y, some_data(0), cvals)    # first image on screen
cb = plt.colorbar();

# animation function
def animate(i):
    global cont
    z = some_data(i)
    for c in cont.collections:
        c.remove()  # removes only the contours, leaves the rest intact
    cont = plt.contourf(x, y, z, cvals)
    plt.title('t = %i:  %.2f' % (i,z[5,5]))
    return cont

anim = animation.FuncAnimation(fig, animate, frames=Nt, blit=True, repeat=True);